<a target="_blank" href="https://colab.research.google.com/github/UpstageAI/cookbook/blob/main/Solar-Fullstack-LLM-101/09_9_RAG_API.ipynb">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Upstage RAG API (experimental)



## Environment

In [1]:
!pip install -qU langchain-upstage langchain python-dotenv openai rich

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
goldenverba 2.1.0 requires langchain-text-splitters==0.2.2, but you have langchain-text-splitters 0.3.2 which is incompatible.
goldenverba 2.1.0 requires python-dotenv==1.0.0, but you have python-dotenv 1.0.1 which is incompatible.
goldenverba 2.1.0 requires tiktoken==0.6.0, but you have tiktoken 0.8.0 which is incompatible.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


### Environment variables

Set up environment variables 
* UPSTAGE_API_KEY


In [2]:
# @title set API key
from pprint import pprint
from rich import print as rprint

import os

import warnings

warnings.filterwarnings("ignore")

if "google.colab" in str(get_ipython()):
    # Running in Google Colab. Please set the UPSTAGE_API_KEY in the Colab Secrets
    from google.colab import userdata

    os.environ["UPSTAGE_API_KEY"] = userdata.get("UPSTAGE_API_KEY")
else:
    # Running locally. Please set the UPSTAGE_API_KEY in the .env file
    from dotenv import load_dotenv

    load_dotenv()

assert (
    "UPSTAGE_API_KEY" in os.environ
), "Please set the UPSTAGE_API_KEY environment variable"

In [9]:
rag_base_url = "https://rag.toy.x.upstage.ai/"
#rag_base_url = "http://localhost:8000/"

# Plase chose your model name
rag_model_name = "sung-pro-1009"

In [10]:
from openai import OpenAI


client = OpenAI(base_url=rag_base_url, api_key=os.environ["UPSTAGE_API_KEY"])

In [11]:

print("Building/Rebuilding the index...")
directory = "pdfs"
# Upload all files in the pdfs folder
for file in os.listdir(directory):
    file_path = os.path.join(directory, file)
    with open(file_path, "rb") as f:
        file_response = client.files.create(
            file=f, purpose="assistants", extra_body={"model_name": rag_model_name}
        )
        rprint(file_response)

print("Index build/rebuild submission completed.")

public_model_name = file_response.public_model_name
print("Public Mode Name:", public_model_name)

Building/Rebuilding the index...


FileObject(
    id='file-6',
    bytes=418872,
    created_at=1730856387,
    filename='solar_paper.pdf',
    object='file',
    purpose='assistants',
    status='queued',
    status_details=None,
    model_name='sung-pro-1008',
    public_model_name='367ebd586fee0553547b'
)

FileObject(
    id='file-7',
    bytes=137425,
    created_at=1730856387,
    filename='solar_sample.pdf',
    object='file',
    purpose='assistants',
    status='queued',
    status_details=None,
    model_name='sung-pro-1008',
    public_model_name='367ebd586fee0553547b'
)

FileObject(
    id='file-8',
    bytes=878624,
    created_at=1730856388,
    filename='docai.pdf',
    object='file',
    purpose='assistants',
    status='queued',
    status_details=None,
    model_name='sung-pro-1008',
    public_model_name='367ebd586fee0553547b'
)

FileObject(
    id='file-9',
    bytes=4432693,
    created_at=1730856389,
    filename='kim-tse-2008.pdf',
    object='file',
    purpose='assistants',
    status='queued',
    status_details=None,
    model_name='sung-pro-1008',
    public_model_name='367ebd586fee0553547b'
)

Index build/rebuild submission completed.
Public Mode Name: 367ebd586fee0553547b


In [13]:
# It will take a few minutes to build the index
# You may get InternalServerError: 
# Error code: 503 - Service Not Ready
# If you do, just wait for indexing to complete and try again

# Import necessary libraries
from langchain_upstage import ChatUpstage as Chat

# Initialize the ChatUpstage model
# Use the previously defined rag_model_name and rag_base_url
chat = Chat(model=rag_model_name, base_url=rag_base_url)

# Send a query to the chat model
question = "What's the Depth UpScaling for LLM?"
response = chat.invoke(question)

# Print the response content in a formatted way
rprint(response)

InternalServerError: Internal Server Error

In [ ]:
# It will take a few minutes to build the index
# You may get InternalServerError: 
# Error code: 503 - Service Not Ready
# If you do, just wait for indexing to complete and try again

# Import necessary libraries
from langchain_upstage import ChatUpstage as Chat

# Initialize the ChatUpstage model
# Use the previously defined rag_model_name and rag_base_url
chat = Chat(model=rag_model_name, base_url=rag_base_url, streaming=True)

# Send a query to the chat model
question = "What's the Depth UpScaling for LLM?"
response = chat.invoke(question)

# Print the response content in a formatted way
rprint(response)

In [ ]:
# Langchain Streaming
# https://github.com/pinecone-io/examples/blob/master/learn/generation/langchain/handbook/09-langchain-streaming/09-langchain-streaming.ipynb
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler


chat = Chat(model=rag_model_name, base_url=rag_base_url,
    streaming=True,  # ! important
    callbacks=[StreamingStdOutCallbackHandler()]  # ! important
)

from langchain.schema import HumanMessage

# create messages to be passed to chat LLM
messages = [HumanMessage(content="What's the Depth UpScaling for LLM? Describe it in detail.")]

response = chat.invoke(messages)


The Depth UpScaling (DUS) for LLM is a method presented in the document "SOLAR 10.7B: Scaling Large Language Models with Simple yet Effective Depth Up-Scaling." DUS is a strategy for scaling LLMs and it consists of two main components: depthwise scaling and continued pre-training.

Depthwise scaling is the process of increasing the depth of the LLM by adding more layers to the model architecture. This is done by duplicating the base model's layers, which allows for the creation of a larger model with more parameters. In the case of SOLAR 10.7B, the model has 10.7 billion parameters, which is achieved through depthwise scaling.

After depthwise scaling, the scaled model undergoes continued pre-training. This process involves further training the model on a large-scale dataset to improve its performance and adapt it to the increased model size. Continued pre-training helps the model to better utilize its additional parameters and improve its generalization ability.

DUS is contrasted to 

# Upstage RAG-API Configuration Options

The `Chat` object is initialized with several options to customize its behavior:

1. `api_key`: The API key for authentication. Get from https://console.upstage.ai/

2. `model`: The name of the RAG (Retrieval-Augmented Generation) model to be used.

3. `base_url`: The base URL for the RAG API.

4. `extra_body`: A dictionary containing additional parameters for the API request:

   - `hybrid_search` (bool): When set to `True`, enables a combination of semantic and keyword-based search for improved retrieval accuracy.
   
   - `contextual_query` (bool): If `True`, the system considers the context of previous interactions when processing the current query.
   
   - `contextual_chunk` (bool): When enabled, the system retrieves and processes information in context-aware chunks, potentially improving relevance.
   
   - `knowledge_graph` (bool): If set to `True`, the system utilizes a knowledge graph to enhance understanding and connections between concepts.
   
   - `kv_pairs` (bool): When enabled, the system extracts and utilizes key-value pairs from the input, which can be useful for structured data processing.
   
   - `llm_model_name` (str): Specifies the name of the language model to be used, in this case "solar-pro".

These options allow for fine-tuning of the RAG system's behavior, potentially improving the relevance and accuracy of responses based on the specific use case and requirements.

In [ ]:
# Import necessary libraries
from langchain_upstage import ChatUpstage as Chat

# Initialize the ChatUpstage model
# Use the previously defined rag_model_name and rag_base_url
chat = Chat(
    api_key=os.environ["UPSTAGE_API_KEY"],
    model=rag_model_name,
    base_url=rag_base_url,
    extra_body={
        "hybrid_search": True,
        "contextual_query": True,
        "contextual_chunk": True,
        "knowledge_graph": True,
        "kv_pairs": True,
        "llm_model_name": "solar-pro",
    },
)

# Send a query to the chat model
question = "What's the DUS?"
response = chat.invoke(question)

# Print the response content in a formatted way
rprint(response)

AIMessage(
    content='The DUS is a new method for scaling up large language models (LLMs). It is compatible with easy-to-use
LLM frameworks like HuggingFace and applicable to all transformer architectures. It has been used to create SO-LAR 
10.7B, an LLM with 10.7 billion parameters that outperforms existing models, and SOLAR 10.7B-Instruct, a variant 
fine-tuned for complex instructions that outperforms other models in various evaluation metrics.',
    additional_kwargs={'refusal': None},
    response_metadata={
        'token_usage': None,
        'model_name': 'sung-pro-1001',
        'system_fingerprint': None,
        'finish_reason': None,
        'logprobs': None
    },
    id='run-7acffdab-6565-45b9-8c05-5406e9a68de6-0'
)

In [164]:
rprint(chat.get_num_tokens(question))
print(question)

13

What's the Depth UpScaling for LLM?


In [15]:
from openai import OpenAI

# Initialize the OpenAI client with custom base URL and API key
client = OpenAI(
    base_url=rag_base_url,  # Using the custom RAG API base URL
    api_key=os.environ[
        "UPSTAGE_API_KEY"
    ],  # Using the API key from environment variables
)

# Send a request to the RAG model
response = client.chat.completions.create(
    model=rag_model_name,  # Using the previously defined RAG model name
    messages=[{"role": "user", "content": "What's Solar 10.7B LLM MMLU?"}],  # The user's query
    stream=True,  # Disable streaming for synchronous response
    extra_body={
        "hybrid_search": True,  # Enable hybrid search for better results
        "contextual_chunk": False,
        "contextual_query": True,
        "knowledge_graph": False,
        "kv_pairs": False,
        "verbose": True,  # Request verbose output for debugging
    },
)

# Print the model's response
rprint(response)

ChatCompletion(
    id='_',
    choices=[
        Choice(
            finish_reason=None,
            index=0,
            logprobs=None,
            message=ChatCompletionMessage(
                content='The following indexed files are being used for this query: solar_paper.pdf, 
solar_sample.pdf, docai.pdf, kim-tse-2008.pdf\n\nSolar 10.7B is a large language model (LLM) with 10.7 billion 
parameters, demonstrating superior performance in various natural language processing (NLP) tasks. In the MMLU 
(Multi-Domain Evaluation) framework, it achieved an average score of 69.83, as shown in Table 2 of the provided 
content.',
                refusal=None,
                role='assistant',
                function_call=None,
                tool_calls=None
            )
        )
    ],
    created=1728237594.0,
    model='sung-pro-1008',
    object='chat.completion',
    service_tier=None,
    system_fingerprint=None,
    usage=[
        CompletionUsage(
            completion_tokens=None,
            prompt_tokens=None,
            total_tokens=None,
            completion_tokens_details=None,
            prompt_tokens_details=None,
            id=None,
            metadata={
                'total_pages': 13,
                'filename': 'solar_paper',
                'original_chunk': 'research and practical applications, fostering\nwider accessibility and utility 
in various fields.• Superior Performance Across Diverse\nBenchmarks: SOLAR 10.7B excels in var-\nious benchmarks, 
outperforming established\nmodels like Llama 2 and Mistral 7B in reason-\ning, mathematics, and the MMLU 
framework.• Advancement in Instruction-Following Ca-\npabilities: The introduction of SOLAR 10.7B-\nInstruct, a 
variant fine-tuned for enhanced\ninstruction-following abilities, marks a sig-\nnificant improvement in the model’s
ability to\nunderstand and execute complex instructions.Sanghoon Kim, Dahyun Kim, Chanjun Park,\nWonsung Lee, Wonho
Song, Yunsu Kim and\nHyeonwoo Kim contributed equally to this paper.\nSanghoon Kim led the Foundation Model 
part,\nwith Dahyun Kim, Wonho Song, Yunsu Kim, and\nHyeonwoo Kim. Chanjun Park led the Data and\nEvaluation 
(Data-Centric LLM) part, with Yungi\nKim, Jihoo Kim, Changbae Ahn, Seonghoon Yang,\nSukyung Lee, and Hyunbyung 
Park. Wonsung Lee',
                'is_contextual_chunk': False,
                '_id': 'f6a2b9ec-0af7-4ba0-9936-d1d7f3704f21',
                '_collection_name': 
'97081dab6dca79d677fe6cc3b1c29a4d8c722bb418b504fad71eb75831f649fb_sung-pro-1008'
            },
            page_content='research and practical applications, fostering\nwider accessibility and utility in 
various fields.• Superior Performance Across Diverse\nBenchmarks: SOLAR 10.7B excels in var-\nious benchmarks, 
outperforming established\nmodels like Llama 2 and Mistral 7B in reason-\ning, mathematics, and the MMLU 
framework.• Advancement in Instruction-Following Ca-\npabilities: The introduction of SOLAR 10.7B-\nInstruct, a 
variant fine-tuned for enhanced\ninstruction-following abilities, marks a sig-\nnificant improvement in the model’s
ability to\nunderstand and execute complex instructions.Sanghoon Kim, Dahyun Kim, Chanjun Park,\nWonsung Lee, Wonho
Song, Yunsu Kim and\nHyeonwoo Kim contributed equally to this paper.\nSanghoon Kim led the Foundation Model 
part,\nwith Dahyun Kim, Wonho Song, Yunsu Kim, and\nHyeonwoo Kim. Chanjun Park led the Data and\nEvaluation 
(Data-Centric LLM) part, with Yungi\nKim, Jihoo Kim, Changbae Ahn, Seonghoon Yang,\nSukyung Lee, and Hyunbyung 
Park. Wonsung Lee',
            type='Document'
        ),
        CompletionUsage(
            completion_tokens=None,
            prompt_tokens=None,
            total_tokens=None,
            completion_tokens_details=None,
            prompt_tokens_details=None,
            id=None,
            metadata={
                'total_pages': 13,
                'filename': 'solar_paper',
                'original_chunk': 'Wen, and Jiawei Han. 2023. D

In [ ]:
# Contextual Query

from openai import OpenAI

# Initialize the OpenAI client with custom base URL and API key
client = OpenAI(
    base_url=rag_base_url,  # Using the custom RAG API base URL
    api_key=os.environ[
        "UPSTAGE_API_KEY"
    ],  # Using the API key from environment variables
)
# Send a request to the RAG model
response = client.chat.completions.create(
    model=rag_model_name,  # Using the previously defined RAG model name
    messages=[
        {"role": "user", "content": "What's the benefits of Document AI?"},
        {"role": "assistant", "content": "It can help you understand the Document AI"},
        {"role": "user", "content": "How about its features?"},  # The user's query
    ],
    stream=False,  # Disable streaming for synchronous response
    extra_body={
        "contextual_query": True,
        "contextual_chunk": False,
        "knowledge_graph": False,
        "kv_pairs": False,
        "verbose": True,  # Request verbose output for debugging
    },
)

# Print the model's response
rprint(response)

ChatCompletion(
    id='_',
    choices=[
        Choice(
            finish_reason=None,
            index=0,
            logprobs=None,
            message=ChatCompletionMessage(
                content='The following indexed files are being used for this query: solar_paper.pdf, 
solar_sample.pdf, docai.pdf, kim-tse-2008.pdf\n\nThe features of Document AI are:\n\n1. "Layout Analyzer: Extracts 
layouts, tables, and figures from any document."\n2. "Key Information Extractor: Extracts key information from 
target documents."\n3. "Document OCR: Extracts all text from any document."\n\nThese features have resulted in 
higher accuracy (95%-98% accuracy rate) compared to previous manual human work for document processing in various 
enterprise use cases.',
                refusal=None,
                role='assistant',
                function_call=None,
                tool_calls=None
            )
        )
    ],
    created=1727891103.0,
    model='sung-pro-1001',
    object='chat.completion',
    service_tier=None,
    system_fingerprint=None,
    usage=[
        CompletionUsage(
            completion_tokens=None,
            prompt_tokens=None,
            total_tokens=None,
            completion_tokens_details=None,
            prompt_tokens_details=None,
            id=None,
            metadata={
                'total_pages': 1,
                'filename': 'docai',
                'original_chunk': 'Document AI: Digitize anythingUpstage Document AI is comprised of three 
solutions:● Layout Analyzer: Extracts layouts, tables, and figures from any document.\n● Key Information Extractor:
Extracts key information from target documents.\n● Document OCR: Extracts all text from any document.In various 
enterprise use cases, Upstage Document AI resulted in higher accuracy\n(95%-98% accuracy rate) compared to previous
manual human work for document\nprocessing. Due to its strong performance, the solution has been heavily adopted by
various\nleading insurance providers, yielding significant productivity gains and operating cost\nreductions.Please
find customer testimonials, key customers, and customer success stories below.[Key customers]',
                'is_contextual_chunk': False,
                '_id': 'bf98c39f-4f74-4eda-9d62-b411fd22d25d',
                '_collection_name': 
'97081dab6dca79d677fe6cc3b1c29a4d8c722bb418b504fad71eb75831f649fb_sung-pro-1001',
                'contextual_query': '"What are the features of Document AI?"'
            },
            page_content='Document AI: Digitize anythingUpstage Document AI is comprised of three solutions:● 
Layout Analyzer: Extracts layouts, tables, and figures from any document.\n● Key Information Extractor: Extracts 
key information from target documents.\n● Document OCR: Extracts all text from any document.In various enterprise 
use cases, Upstage Document AI resulted in higher accuracy\n(95%-98% accuracy rate) compared to previous manual 
human work for document\nprocessing. Due to its strong performance, the solution has been heavily adopted by 
various\nleading insurance providers, yielding significant productivity gains and operating cost\nreductions.Please
find customer testimonials, key customers, and customer success stories below.[Key customers]',
            type='Document'
        ),
        CompletionUsage(
            completion_tokens=None,
            prompt_tokens=None,
            total_tokens=None,
            completion_tokens_details=None,
            prompt_tokens_details=None,
            id=None,
            metadata={
                'total_pages': 16,
                'filename': 'kim-tse-2008',
                'original_chunk': '[42] M.D. Penta, S. Gradara, and G. Antoniol, “Traceability Recovery\nin RAD 
Software Systems,” Proc. 10th IEEE Int’l Workshop Program\nComprehension, pp. 207-216, 2002.\n[43] B. Raskutti, 
H.L. Ferra´ , and A. Kowalczyk, “Second-Order\nFeatures for Maximizing Text Classification Performance,” 
Proc.\n12th European Conf.

In [ ]:
from openai import OpenAI

# Initialize the OpenAI client with custom base URL and API key
client = OpenAI(
    base_url=rag_base_url,  # Using the custom RAG API base URL
    api_key=os.environ[
        "UPSTAGE_API_KEY"
    ],  # Using the API key from environment variables
)

# Send a request to the RAG model
response = client.chat.completions.create(
    model=rag_model_name,  # Using the previously defined RAG model name
    messages=[{"role": "user", "content": "Explain Document AI"}],  # The user's query
    stream=False,  # Disable streaming for synchronous response
    extra_body={
        "hybrid_search": True,  # Enable hybrid search for better results
        "contextual_query": True,  # Enable contextual querying
        "knowledge_graph": True,
        "kv_pairs": True,
        "verbose": True,  # Request verbose output for debugging
    },
)

# Print the model's response
rprint(response)

ChatCompletion(
    id='_',
    choices=[
        Choice(
            finish_reason=None,
            index=0,
            logprobs=None,
            message=ChatCompletionMessage(
                content='The following indexed files are being used for this query: solar_paper.pdf, 
solar_sample.pdf, docai.pdf, kim-tse-2008.pdf\n\nBased on the provided context, key-value pairs, and knowledge 
graph, I can explain "Document AI" as follows:\n\nThe main topic of this document is "Upstage Document AI: Digitize
anything". Document AI is described as providing three solutions: "Layout Analyzer", "Key Information Extractor", 
and "Document OCR". Layout Analyzer extracts layouts, tables, and figures from any document. Key Information 
Extractor extracts key information from target documents. Document OCR extracts all text from any document. In 
enterprise use cases, Document AI achieved an accuracy rate of 95-98%, leading to significant productivity gains 
and operating cost reductions for leading insurance providers.\n\nHere are some direct quotes from the 
context:\n\n1. "This chunk introduces the main document and its three solutions, Layout Analyzer, Key Information 
Extractor, and Document OCR, and highlights its effectiveness in enterprise use cases, achieving 95-98% accuracy 
rates, leading to significant adoption by leading insurance providers and resulting in productivity gains and 
operating cost reductions."\n2. "Upstage Document AI: Digitize anything is comprised of the following 
solutions:●Layout Analyzer: Extracts layouts, tablescs, and figures from any document●Key Information Extractor: 
Extracts key information from target documents●Document OCR: Extracts all text from any document"\n\nAnd here are 
some key-value pairs and knowledge graph entries relevant to this answer:\n\n- [{\'key\': \'Document Title\', 
\'value\': \'Upstage Document AI: Digitize anything\'}, {\'key\': \'Solution 1\', \'value\': \'Layout Analyzer\'}, 
{\'key\': \'Solution 1 Description\', \'value\': \'Extracts layouts, tables, and figures from any document\'}, 
{\'key\': \'Solution 2\', \'value\': \'Key Information Extractor\'}, {\'key\': \'Solution 2 Description\', 
\'value\': \'Extracts key information from target documents\'}, {\'key\': \'Solution 3\', \'value\': \'Document 
OCR\'}, {\'key\': \'Solution 3 Description\', \'value\': \'Extracts all text from any document\'}, {\'key\': 
\'Benefits\', \'value\': \'Significant productivity gains and operating cost reductions\'}, {\'key\': 
\'customer_type\', \'value\': \'Leading insurance providers\'}]\n- {\'knowledge_graph\': {\'type\': \'Document\', 
\'attributes\': {\'title\': \'Upstage Document AI: Digitize anything\'}, \'relationships\': {\'solutions\': 
[{\'name\': \'Layout Analyzer\', \'description\': \'Extracts layouts, tables, and figures from any document\', 
\'type\': \'Solution\'}, {\'name\': \'Key Information Extractor\', \'description\': \'Extracts key information from
target documents\', \'type\': \'Solution\'}, {\'name\': \'Document OOC\', \'description\': \'Extracts all text from
any document\', \'type\': \'Solution\'}], \'performance\': {\'accuracyRate\': \'95-98%\', \'comparison\': \'Higher 
accuracy than previous manual human work\', \'customerType\': \'Leading insurance providers\', \'benefits\': 
\'Significant productivity gains and operating cost reductions\'}]}}}',
                refusal=None,
                role='assistant',
                function_call=None,
                tool_calls=None
            )
        )
    ],
    created=1727891121.0,
    model='sung-pro-1001',
    object='chat.completion',
    service_tier=None,
    system_fingerprint=None,
    usage=[
        CompletionUsage(
            completion_tokens=None,
            prompt_tokens=None,
            total_tokens=None,
            completion_tokens_details=None,
            prompt_tokens_details=None,
            id=None,
            metadata={
                'total_pages': 1,
                'filename':

In [ ]:
# Initialize the chat model using the public model name (accessible to everyone)
chat = Chat(model=public_model_name, base_url=rag_base_url)

# Define the question about Document AI and its usage
question = "What's Document AI? How can it be used?"

# Invoke the chat model with the question
response = chat.invoke(question)
rprint(response)

AIMessage(
    content='The document AI, as described in the context provided, is a product called "Upstage Document AI: 
Digitize anything." It consists of three solutions: "Layout Analyzer," "Key Information Extractor," and "Document 
OCR." It can be used to extract layouts, tables, and figures from any document, extract key information from target
documents, and extract all text from any document, respectively.\n\nIn enterprise use cases, Upstage Document AI 
has achieved higher accuracy compared to previous manual human work, leading to significant adoption by leading 
insurance providers and resulting in productivity gains and operating cost reductions.',
    additional_kwargs={'refusal': None},
    response_metadata={
        'token_usage': None,
        'model_name': '1c9fba8131d5829120ff',
        'system_fingerprint': None,
        'finish_reason': None,
        'logprobs': None
    },
    id='run-c6e2b043-0dee-497b-b44d-0a5c836fbbc6-0'
)

In [ ]:
# RAG API with history
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage
from langchain_core.output_parsers import StrOutputParser

prompt_template = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder("chat_history"),
        ("user", "{input}"),
    ]
)
chat = Chat(model=public_model_name, base_url=rag_base_url)

chain = prompt_template | chat

questions = [
    "What's the benefits of Document AI?",
    "How about its features?",
    "Why we need it?",
]

chat_history = []

for question in questions:
    print("Question: ", question)

    human_message = HumanMessage(content=question)
    response = chain.invoke({"input": question, "chat_history": chat_history})
    rprint(response)
    chat_history.append(human_message)
    chat_history.append(response)

# Print the chat history
print("Chat History:")
for chat in chat_history:
    rprint(chat)

Question:  What's the benefits of Document AI?


AIMessage(
    content='The benefit of Upstage Document AI, as stated in the provided information, is "Significant 
productivity gains and operating cost reductions."',
    additional_kwargs={'refusal': None},
    response_metadata={
        'token_usage': None,
        'model_name': '1c9fba8131d5829120ff',
        'system_fingerprint': None,
        'finish_reason': None,
        'logprobs': None
    },
    id='run-e080b83c-702f-4101-9ca3-6d2f9ed712ea-0'
)

Question:  How about its features?


AIMessage(
    content='The question asks for information about the features of Document AI, but the provided context, 
key-value pairs, knowledge graph, and all other instructions are about Upstage Document AI and its specific tools: 
Layout Analyzer, Key Information Extractor, and Document OCR. These tools have the following features:\n\n1. Layout
Analyzer: "Extracts layouts, tables, and figures from any document"\n2. Key Information Extractor: "Extracts key 
information from target documents"\n3. Document OCR: "Extracts all text from any document"\n\nThe provided 
information does not specify any general features of Document AI as a whole, only the features of its specific 
tools.',
    additional_kwargs={'refusal': None},
    response_metadata={
        'token_usage': None,
        'model_name': '1c9fba8131d5829120ff',
        'system_fingerprint': None,
        'finish_reason': None,
        'logprobs': None
    },
    id='run-93da3c16-a456-47aa-8675-37b2f9df56d4-0'
)

Question:  Why we need it?


...

AIMessage(
    content='The benefit of Upstage Document AI, as stated in the provided information, is "Significant 
productivity gains and operating cost reductions."',
    additional_kwargs={'refusal': None},
    response_metadata={
        'token_usage': None,
        'model_name': '1c9fba8131d5829120ff',
        'system_fingerprint': None,
        'finish_reason': None,
        'logprobs': None
    },
    id='run-e080b83c-702f-4101-9ca3-6d2f9ed712ea-0'
)

HumanMessage(content='How about its features?', additional_kwargs={}, response_metadata={})

AIMessage(
    content='The question asks for information about the features of Document AI, but the provided context, 
key-value pairs, knowledge graph, and all other instructions are about Upstage Document AI and its specific tools: 
Layout Analyzer, Key Information Extractor, and Document OCR. These tools have the following features:\n\n1. Layout
Analyzer: "Extracts layouts, tables, and figures from any document"\n2. Key Information Extractor: "Extracts key 
information from target documents"\n3. Document OCR: "Extracts all text from any document"\n\nThe provided 
information does not specify any general features of Document AI as a whole, only the features of its specific 
tools.',
    additional_kwargs={'refusal': None},
    response_metadata={
        'token_usage': None,
        'model_name': '1c9fba8131d5829120ff',
        'system_fingerprint': None,
        'finish_reason': None,
        'logprobs': None
    },
    id='run-93da3c16-a456-47aa-8675-37b2f9df56d4-0'
)

HumanMessage(content='Why we need it?', additional_kwargs={}, response_metadata={})

AIMessage(
    content='Based on the provided context, key-value pairs, and knowledge graph, the reason for needing Document 
AI is not explicitly stated. However, I can infer some potential reasons based on the information given. The 
context mentions that Upstage Document AI: Digitize anything has been adopted by leading insurance providers due to
its higher accuracy rates compared to previous manual human work, leading to significant productivity gains and 
operating cost reductions. Therefore, Document AI may be needed to improve the accuracy of document processing, 
increase productivity, and reduce operating costs.',
    additional_kwargs={'refusal': None},
    response_metadata={
        'token_usage': None,
        'model_name': '1c9fba8131d5829120ff',
        'system_fingerprint': None,
        'finish_reason': None,
        'logprobs': None
    },
    id='run-eb426770-3337-4785-ae6d-9dbcf35a1327-0'
)

In [ ]:
# Public mode name is read-only
rprint(f"Try to upload a file to the public model...{public_model_name}")

# Expect 400 BadRequest Error
# If you want to upload a file to the public model, you need to use the private model name

try:
    file_response = client.files.create(
        file=open("pdfs/solar_paper.pdf", "rb"),
        purpose="assistants",
        extra_body={"model_name": public_model_name},
    )
except Exception as e:
    rprint(e)

Try to upload a file to the public model...1c9fba8131d5829120ff

Error code: 400 - {'detail': "Invalid model name. Please use only alphanumeric characters, hyphens, and 
underscores. Example: 'my-model-123'"}

In [ ]:
# Delete the model
response = client.models.delete(rag_model_name)
print(response)